In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

from os import path
path = '/content/drive/MyDrive/csv/Shopping_CustomerData (1).csv'
df = pd.read_csv(path)

df.describe()

In [ ]:
df_new = df[["CustomerAge","SpendingScore"]] #Mensclining data yang akan di analisi
df_new.head

In [ ]:
df_new.describe()

In [ ]:
df_new.isnull().sum() #Mencari apakah ada data yang null

In [ ]:
df.plot(kind='scatter', x='CustomerAge', y='SpendingScore')
#plt.scatter(df_new.iloc[:,0], df_new.iloc[:,1])
plt.xlabel('Usia Pembeli')
plt.ylabel('Pendapatan Skor')
plt.title('uclustering data')
plt.show()

In [ ]:
#Step 1
k = 3 #asumsikan kita akan mencari cluster
np.random.seed(40)

#disini kita akan mendapatkan 3 point random centroid
random_centroids=[]
for i in range(1,k+1):
  random_centroids.append([np.random.randint(1,100), np.random.randint(1,100)])
  print('Randomly selected points as random_centroids:', random_centroids)

In [ ]:
#plotting random centroid tiap segmen
plt.figure(figsize=(5, 5))
plt.scatter(df_new.iloc[:,0], df_new.iloc[:,1], color='cyan')
length = len(random_centroids)
colors1=['r','g','b','cyan','yellow','black']
for i in range(length):
  plt.scatter(*random_centroids[i],color=colors1[i])
plt.xlabel('CustomerAge')
plt.ylabel('SpendingScore')
plt.show()

In [ ]:
#Langkah kedua

#Here, we are calculating the distance between the datapoints and our centroids and assigning each data-point to the closest cluster
def assignment(df_new,random_centroids):

  for i in range(length) :
    df_new['Distance from Centroid {}'.format(i)]=(np.sqrt((df_new.iloc[:,0] - random_centroids[i][0]) ** 2 + (df_new.iloc[:,1] - random_centroids[i][1]) ** 2))
   
  list1=[]
  list2=[]
  for a,b,c in zip(df_new['Distance from Centroid 0'],df_new['Distance from Centroid 1'],df_new['Distance from Centroid 2']):
    d = min(a,b,c)
   
    if d == a:
      list1.append(0)
      list2.append('r')
    elif d == b:
      list1.append(1)
      list2.append('g')
    else:
      list1.append(2)
      list2.append('b')
  df_new['Closest_Centroid'] = list1
  df_new['Color']=list2
  return df_new

In [ ]:
df_new = assignment(df_new, random_centroids)
df_new.head()

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(df_new.iloc[:,0],df_new.iloc[:,1],color =df_new['Color'],alpha = 0.5,edgecolor = 'k')
for i in range(length):
  plt.scatter(*random_centroids[i],color=colors1[i])

In [ ]:
#Penempatan Cluster
plt.figure(figsize=(5,5))
plt.scatter(df_new.iloc[:,0],df_new.iloc[:,1],color =df_new['Color'],alpha = 0.5,edgecolor = 'k')
for i in range(length):
  plt.scatter(*random_centroids[i],color=colors1[i])

In [ ]:
def update(parameter):
  for i in range(length):
    random_centroids[i][0] = np.mean(df_new[df_new['Closest_Centroid'] == i]['CustomerAge'])
    random_centroids[i][0] = np.mean(df_new[df_new['Closest_Centroid'] == i]['SpendingScore'])
    return parameter

In [ ]:
random_centroids = update(random_centroids)
random_centroids

In [ ]:
#Step last
#mencari titik centroid paling tepat
df_new = assignment(df_new, random_centroids)
df_new.head()
plt.figure(figsize=(5, 5))
plt.scatter(df_new.iloc[:,0], df_new.iloc[:,1], color=df_new['Color'], alpha=0.5, edgecolor='k')
for i in range(length):
  plt.scatter(*random_centroids[i], color=colors1[i])
plt.show()

In [ ]:
#step 4
#update centroid sampai perhitungan mean berhenti

count=1
while True:
  old_random_centroids = np.round(random_centroids)
  print('Old Centroid',old_random_centroids)
  count+=1
  random_centroids = update(random_centroids)

  new_random_centroids = np.round(random_centroids)
  print('New Centroid',new_random_centroids)

  df_new = assignment(df_new, random_centroids)
  result=np.allclose(old_random_centroids,new_random_centroids)
  print(result)
  if result == True:
    break
    
print(count)

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(df_new.iloc[:,0], df_new.iloc[:,1], color=df_new['Color'], alpha=0.5, edgecolor='k')
plt.show()

In [ ]:
#membaca 2 feature
df = df[["CustomerAge","SpendingScore"]]
df.head()

In [ ]:
#asumsikan nilai k
kmeans = KMeans(n_clusters=3)#Membuat objek K-means
kmeans.fit(df)#Fiting the model

In [ ]:
#generate label tiap point
labels = kmeans.predict(df)
labels

In [ ]:
#menampilkan data centroid tiap cluster
centroids = kmeans.cluster_centers_
centroids

In [ ]:
#melihat inersialk-means
kmeans.inertia_

In [ ]:
plt.figure(figsize=(10, 5))
colmap = {1:'y',2:'g',3:'b',4:'r',5:'c'} #penentuan warna plot
colors = map(lambda x: colmap[x+1], labels)
print(colors)
colors1=list(colors)
plt.scatter(df['CustomerAge'], df["SpendingScore"], color=colors1, alpha=0.5)
for idx, centroid in enumerate(centroids):
  plt.scatter(*centroid, color=colmap[idx+1])
plt.xlabel('usia')
plt.ylabel('pendapatan skor')
plt.title('plot ter-clusterisasi k-means')
plt.show()

In [ ]:
inertia_list = []
for num_clusters in np.arange(1, 21):
  kmeans =KMeans(n_clusters=num_clusters)
  kmeans.fit(df)
  inertia_list.append(kmeans.inertia_)

In [ ]:
#plotting menggunakan elbow curve
plt.figure(figsize=(10, 5))
plt.plot(np.arange(1,21), inertia_list, color='blue')
plt.scatter(np.arange(1, 21), inertia_list, color='red')
plt.grid(True)
plt.xlabel('Nilai dari k')
plt.ylabel('nilai inertia')
plt.title('Elbow curve')
plt.show()